# Test Plotly/Mapbox Integration

In [3]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd
import pytz
pacific = pytz.timezone('US/Pacific')

mapbox_access_token = 'pk.eyJ1Ijoic3Vwc2NpZW50aXN0IiwiYSI6ImNqaXo0dnkxZjAzZWEzdnFlcWVleTB1aXMifQ.XGQdl8PjrnsCDTJ_v3j0Sg'

### Grab Smartfin data from Bike to Surf Month

In [4]:
# Import CSV as DataFrame
smartfin_df = pd.read_csv('bikeToSurf_DataFrame.csv', parse_dates=[0])
smartfin_df.set_index('Sample Time', inplace = True)

# Get index, convert to local time, then shove back into index
daytimes = smartfin_df.index.get_level_values(-1)
daytimes = daytimes.tz_localize(pytz.utc).tz_convert(pacific)

smartfin_df.loc[:, 'Sample Time'] = daytimes
smartfin_df.set_index('Sample Time', inplace = True)
smartfin_df.head()

,Time_x,Temperature 1,Calibrated Temperature 1,Temperature 1 Stable,Temperature 2,Calibrated Temperature 2,Temperature 2 Stable,salinity,Calibrated Salinity,Salinity Stable,...,IMU A2,IMU A3,IMU G1,IMU G2,IMU G3,IMU M1,IMU M2,IMU M3,Latitude,Longitude
Sample Time,,,,,,,,,,,,,,,,,,,,,
2018-06-01 08:28:18-07:00,954736941.0,295.0,18.438,0.0,4573.0,18.012,0.0,NaN,NaN,NaN,...,485.708333,41.208333,4.166667,-1.000000,7.125000,177.375000,-26.625000,493.208333,NaN,NaN
2018-06-01 08:28:24-07:00,954742961.0,294.0,18.375,0.0,4561.0,17.965,0.0,NaN,NaN,NaN,...,503.375000,1.458333,-47.208333,12.291667,36.750000,149.666667,-19.250000,490.416667,NaN,NaN
2018-06-01 08:28:30-07:00,954748981.0,294.0,18.375,0.0,4567.0,17.989,0.0,NaN,NaN,NaN,...,519.125000,0.666667,-24.958333,-45.083333,11.208333,179.750000,-16.583333,518.083333,NaN,NaN
2018-06-01 08:28:36-07:00,954755000.0,293.0,18.312,0.0,4568.0,17.993,0.0,NaN,NaN,NaN,...,498.458333,51.958333,13.583333,-10.875000,17.583333,172.250000,-23.333333,493.833333,NaN,NaN
2018-06-01 08:28:42-07:00,954761021.0,293.0,18.312,0.0,4572.0,18.008,0.0,NaN,NaN,NaN,...,488.541667,64.666667,-37.208333,-1.958333,-7.375000,172.666667,-29.166667,484.250000,NaN,NaN


### Import lat/lon data from GPX files
2018-06-13-BikeMap.ipynb contains the script to generate a CSV file from GPX files

In [5]:
garmin_df = pd.read_csv('all_gps_from_garmin.csv')
garmin_df = garmin_df
garmin_df.head()

,Time,Lat,Lon
0,2018-06-09 16:19:05,32.796414,-117.228867
1,2018-06-09 16:19:14,32.796528,-117.228260
2,2018-06-09 16:19:17,32.796553,-117.228056
3,2018-06-09 16:19:21,32.796553,-117.227790
4,2018-06-09 16:19:32,32.796699,-117.227134


### Grab vars from DataFrame to be plotted below

In [6]:
smartfin_lat = smartfin_df.Latitude
smartfin_lon = smartfin_df.Longitude
plot_var = smartfin_df.loc[:, 'Calibrated Temperature 2']

garmin_lat = garmin_df.Lat
garmin_lon = garmin_df.Lon

### Set up the Plotly map with Scattermapbox

In [35]:
data = [
    # GPX data
    go.Scattermapbox( 
        lat = garmin_lat,
        lon = garmin_lon,
        mode = 'lines',
        showlegend = False,
        connectgaps = False,
        hoverinfo='none',
        opacity = 0.5,
        line = dict(
            width = 1,
            color = 'blue')
    ),
    
    # Smartfin data outlines
    go.Scattermapbox( 
        lat=smartfin_lat,
        lon=smartfin_lon,
        mode='markers',
        showlegend = False,
        marker=dict(
            size=20,
            color='black',
            opacity=0.8
        ),
        hoverinfo='none'
    ),
    
    # Smartfin data
    go.Scattermapbox( 
        lat=smartfin_lat,
        lon=smartfin_lon,
        mode='markers',
        showlegend = True,
        name = 'Temperature (C)',
        marker=dict(
            size=17,
            color=plot_var,
            colorscale = 'Viridis',
            cmin = 17.5,
            cmax = 22.5,
            showscale = True,
            opacity=0.6
        ),
        text=plot_var,
        hoverinfo='text'
    ),
    
]

layout = go.Layout(
    title='Smartfin Temperature (C) and Bike/Surf Tracks',
    autosize=True,
    hovermode='closest',
    showlegend = False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=32.875,
            lon=-117.265
        ),
        pitch=0,
        zoom=9,
        style='outdoors'
    ),
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 25,
        t = 30,
        pad=4
    )
)

fig = dict(data=data, layout=layout)
plot_url = plotly.offline.plot(fig, filename='Bike-Mapbox-Plotly.html')

In [36]:
js_div = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')
js_txt_name = 'Bike-Mapbox-Plotly.txt'
with open(js_txt_name, 'w+') as f:
     f.write(js_div)

### TODO:
- fix time (currently in UTC)
- improve hover box text
- add GPX tracks